## Data set

We will be using two of the data sets provided by the Museam of Modern Art (MoMA) in this lecture.  Make sure that you have downloaded each repository

#### Install `git-lfs` on Macs 

In [1]:
!brew install git-lfs

/bin/bash: brew: command not found


#### Install `git-lhs` on `wsl Ubuntu` 

In [1]:
!sudo apt-get install software-properties-common

[sudo] password for fahad: 

In [ ]:
!sudo add-apt-repository ppa:git-core/ppa

In [1]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash

[sudo] password for fahad: 


In [ ]:
!sudo apt-get install git-lfs

#### Install lfs (both systems)

In [ ]:
!git lfs install

#### Clone the collection repo

In [2]:
!rm -rf ./data/MoMA_collection/

In [3]:
!git clone https://github.com/MuseumofModernArt/collection.git ./data/MoMA_collection

Cloning into './data/MoMA_collection'...
remote: Enumerating objects: 505, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 505 (delta 8), reused 30 (delta 5), pack-reused 468
Receiving objects: 100% (505/505), 36.86 MiB | 2.36 MiB/s, done.
Resolving deltas: 100% (102/102), done.
Filtering content: 100% (4/4), 180.53 MiB | 2.38 MiB/s, done.


In [5]:
!ls -al ./data/MoMA_collection/

total 184900
drwxr-xr-x 3 fahad fahad      4096 Sep 19 20:54 .
drwxr-xr-x 4 fahad fahad      4096 Sep 19 20:52 ..
drwxr-xr-x 9 fahad fahad      4096 Sep 19 20:54 .git
-rw-r--r-- 1 fahad fahad        85 Sep 19 20:53 .gitattributes
-rw-r--r-- 1 fahad fahad    999504 Sep 19 20:53 Artists.csv
-rw-r--r-- 1 fahad fahad   3432965 Sep 19 20:53 Artists.json
-rw-r--r-- 1 fahad fahad  61642672 Sep 19 20:53 Artworks.csv
-rw-r--r-- 1 fahad fahad 123226875 Sep 19 20:54 Artworks.json
-rw-r--r-- 1 fahad fahad      4358 Sep 19 20:53 README.md


#### Install the exhibitions repo

In [6]:
!git clone https://github.com/MuseumofModernArt/exhibitions.git ./data/MoMA_exhibitions

Cloning into './data/MoMA_exhibitions'...
remote: Enumerating objects: 33, done.
remote: Total 33 (delta 0), reused 0 (delta 0), pack-reused 33
Unpacking objects: 100% (33/33), 2.26 MiB | 952.00 KiB/s, done.


## The exhibition file gives encoding errors by default

In [8]:
import pandas as pd
exhibitions = pd.read_csv('./data/MoMA_exhibitions/MoMAExhibitions1929to1989.csv')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 405: invalid continuation byte

## Switching encodings fixes the problem

* See [this Stack Overflow question](https://stackoverflow.com/questions/18171739/unicodedecodeerror-when-reading-csv-file-in-pandas-with-python)
* More details on [ISO-8859-1](https://en.wikipedia.org/wiki/ISO/IEC_8859-1)

In [9]:
dat_cols = ['ExhibitionBeginDate', 'ExhibitionEndDate']
exhibitions = pd.read_csv('./data/MoMA_exhibitions/MoMAExhibitions1929to1989.csv', 
                          encoding="ISO-8859-1",
                          parse_dates=dat_cols)
exhibitions.head(2)

,ExhibitionID,ExhibitionNumber,ExhibitionTitle,ExhibitionCitationDate,ExhibitionBeginDate,ExhibitionEndDate,ExhibitionSortOrder,ExhibitionURL,ExhibitionRole,ExhibitionRoleinPressRelease,...,Institution,Nationality,ConstituentBeginDate,ConstituentEndDate,ArtistBio,Gender,VIAFID,WikidataID,ULANID,ConstituentURL
0,2557.0,1,"Cézanne, Gauguin, Seurat, Van Gogh","[MoMA Exh. #1, November 7-December 7, 1929]",1929-11-07,1929-12-07,1.0,moma.org/calendar/exhibitions/1767,Curator,Director,...,NaN,American,1902.0,1981.0,"American, 19021981",Male,109252853.0,Q711362,500241556.0,moma.org/artists/9168
1,2557.0,1,"Cézanne, Gauguin, Seurat, Van Gogh","[MoMA Exh. #1, November 7-December 7, 1929]",1929-11-07,1929-12-07,1.0,moma.org/calendar/exhibitions/1767,Artist,Artist,...,NaN,French,1839.0,1906.0,"French, 18391906",Male,39374836.0,Q35548,500004793.0,moma.org/artists/1053


In [10]:
artists = pd.read_csv("./data/Artists.csv")
artists.head(2)

,ConstituentID,DisplayName,ArtistBio,Nationality,Gender,BeginDate,EndDate,Wiki QID,ULAN
0,1,Robert Arneson,"American, 1930–1992",American,Male,1930,1992,NaN,NaN
1,2,Doroteo Arnaiz,"Spanish, born 1936",Spanish,Male,1936,0,NaN,NaN


In [11]:
artists_schema = get_spark_types(artists, keys=['ConstituentID'])

artists_spark = spark.createDataFrame(artists, schema=artists_schema)
(artists_spark
 .take(5)) >> to_pandas

NameError: name 'get_spark_types' is not defined

In [15]:
from more_dfply import fix_names
from dfply import *
artwork = (pd.read_csv("./data/Artworks.csv")
           >> fix_names
           >> mutate(id = X.index + 1)
          )
artwork.head(2)

,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,...,Circumference_cm,Depth_cm,Diameter_cm,Height_cm,Length_cm,Weight_kg,Width_cm,Seat_Height_cm,Duration_sec,id
0,"Ferdinandsbrücke Project, Vienna, Austria (Ele...",Otto Wagner,6210,"(Austrian, 1841–1918)",(Austrian),(1841),(1918),(Male),1896,Ink and cut-and-pasted painted pages on paper,...,NaN,NaN,NaN,48.6000,NaN,NaN,168.9000,NaN,NaN,1
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,"(French, born 1944)",(French),(1944),(0),(Male),1987,Paint and colored pencil on print,...,NaN,NaN,NaN,40.6401,NaN,NaN,29.8451,NaN,NaN,2


In [16]:
artwork_schema = get_spark_types(artwork, keys=['id'])

artwork_spark = spark.createDataFrame(artwork, schema=artwork_schema)
(artwork_spark
 .take(2)) >> to_pandas

NameError: name 'get_spark_types' is not defined